<a href="https://colab.research.google.com/github/sama774/PSO/blob/main/PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Load dataset
data = load_breast_cancer()
X = data.data[:, :5]
y = data.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
# PSO parameters
num_particles = 3
num_features = 5
max_iter = 10

w = 0.7
c1 = 2
c2 = 2
r1 = 0.5
r2 = 0.3

In [5]:
# Initialize positions (binary)
positions = np.random.randint(0, 2, (num_particles, num_features))

# Initialize velocities
velocities = np.random.uniform(-0.5, 0.5, (num_particles, num_features))

# Personal bests
pbest_positions = positions.copy()
pbest_scores = np.zeros(num_particles)

# Global best
gbest_position = None
gbest_score = 0

In [6]:
def fitness_function(feature_mask):
    # If no feature selected, return 0
    if np.sum(feature_mask) == 0:
        return 0

    X_train_selected = X_train[:, feature_mask == 1]
    X_test_selected = X_test[:, feature_mask == 1]

    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train_selected, y_train)

    predictions = model.predict(X_test_selected)
    return accuracy_score(y_test, predictions)

In [7]:
print("Starting PSO...\n")

for iteration in range(1, max_iter + 1):
    print(f"Iteration {iteration}")

    for i in range(num_particles):
        score = fitness_function(positions[i])

        # Update personal best
        if score > pbest_scores[i]:
            pbest_scores[i] = score
            pbest_positions[i] = positions[i].copy()

        # Update global best
        if score > gbest_score:
            gbest_score = score
            gbest_position = positions[i].copy()

    # Update velocity & position
    for i in range(num_particles):
        velocities[i] = (
            w * velocities[i]
            + c1 * r1 * (pbest_positions[i] - positions[i])
            + c2 * r2 * (gbest_position - positions[i])
        )

        # Update positions (Binary PSO using sigmoid)
        sigmoid = 1 / (1 + np.exp(-velocities[i]))
        positions[i] = (sigmoid > 0.5).astype(int)

    print(f"Global Best: {gbest_position} | Accuracy: {gbest_score:.2f}\n")

Starting PSO...

Iteration 1
Global Best: [1 1 1 1 1] | Accuracy: 0.90

Iteration 2
Global Best: [0 1 1 1 1] | Accuracy: 0.91

Iteration 3
Global Best: [0 1 1 1 1] | Accuracy: 0.91

Iteration 4
Global Best: [0 1 1 1 1] | Accuracy: 0.91

Iteration 5
Global Best: [0 1 1 1 1] | Accuracy: 0.91

Iteration 6
Global Best: [0 1 1 1 1] | Accuracy: 0.91

Iteration 7
Global Best: [0 1 1 1 1] | Accuracy: 0.91

Iteration 8
Global Best: [0 1 1 1 1] | Accuracy: 0.91

Iteration 9
Global Best: [0 1 1 1 1] | Accuracy: 0.91

Iteration 10
Global Best: [0 1 1 1 1] | Accuracy: 0.91



In [8]:
print("Final Result")
print("-----------------------")
print("Best Feature Subset:", gbest_position)
print("Best Accuracy:", round(gbest_score, 2))

selected_features = [i+1 for i, v in enumerate(gbest_position) if v == 1]
print("Selected Feature Numbers:", selected_features)

Final Result
-----------------------
Best Feature Subset: [0 1 1 1 1]
Best Accuracy: 0.91
Selected Feature Numbers: [2, 3, 4, 5]
